In [17]:
from datetime import datetime
from thetadata import ThetaClient
from thetadata_utils import curl
import pandas as pd
from credentials import USERNAME, PASSWORD
client = ThetaClient(username=USERNAME, passwd=PASSWORD, timeout=10)

In [4]:
config = {
    "underlying_ticker": 'spx',
    "options_ticker": 'spxw',
}

In [22]:
path = '/data/td/options/expirations'
expirations = pd.read_csv(f"{path}/{config['options_ticker']}.csv.gz", index=False, compression='gzip')
expirations['date'] = pd.to_datetime(expirations['date'], format='%Y%m%d')
expirations

,date
0,2012-06-01
1,2012-06-08
2,2012-06-22
3,2012-07-06
4,2012-07-13
...,...
1679,2024-11-15
1680,2024-11-29
1681,2024-12-31
1682,2025-03-31


In [27]:
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

for expiration in expirations['date']:
    if start_date.date() <= expiration <= end_date.date():
        strikes = curl('list/strikes', {"root": config['options_ticker'], "exp": expiration.strftime("%Y%m%d")})

        filename = f"/data/thetadata/{config['options_ticker']}/strikes/{expiration}.csv.gz"
        strikes.to_csv(filename, compression='gzip', index=False)
        filename